In [1]:
import os 
import cv2
import numpy as np
from tqdm import tqdm
import glob
import natsort
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

D:\Anaconda3\envs\kimna\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\envs\kimna\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
D:\Anaconda3\envs\kimna\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda3\envs\kimna\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
D:\Anaconda3\envs\kimna\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


### noisy image load

In [17]:
path1 = 'D:/Jupyter/kimna/yolo (2)/2. yolo_v5/data/x_ray/images/test_REALESRGAN/'
temp = natsort.natsorted(os.listdir(path1))

In [18]:
files = []
for i in range(50):
    files.append(path1 + temp[i])

In [19]:
images1 = []
for fl in tqdm(files):
    try:
        image = cv2.imread(fl, cv2.IMREAD_COLOR)
        images1.append(image)
    except:
        break
generate = np.array(images1)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 31.31it/s]


In [20]:
images1[1].shape

(1080, 1920, 3)

### clean image load

In [6]:
path2 = 'D:/Jupyter/kimna/yolo (2)/2. yolo_v5/data/x_ray/images/test_orginal_500/'
temp2 = natsort.natsorted(os.listdir(path2))

In [7]:
files_2 = []
for i in range(50):
    files_2.append(path2 + temp2[i])

In [8]:
images = []
for fl in tqdm(files_2):
    try:
        image = cv2.imread(fl, cv2.IMREAD_COLOR)
        images.append(image)
    except:
        break
clean = np.array(images)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 20.82it/s]


In [9]:
images[1].shape

(1080, 1920, 3)

In [ ]:
for i in range(0,len(clean)):
    
    fig=plt.figure(figsize=(15, 5))
    
    ax = plt.subplot(131)
    plt.imshow(clean[i])
    
    ax = plt.subplot(132)
    plt.imshow(generate[i])
    plt.show()

In [10]:
y_true_list = []
for i in range(50):
    y_true = tf.image.convert_image_dtype(clean[i], tf.float32)
    y_true_list.append(y_true)

In [11]:
y_pred_list2 = []
for i in range(50):
    y_pred = tf.image.convert_image_dtype(generate[i], tf.float32)
    y_pred_list2.append(y_pred)

## PSNR

In [21]:
y_pred_list = []
for i in range(50):
    y_pred = tf.image.convert_image_dtype(generate[i], tf.float32)
    y_pred_list.append(y_pred)

In [22]:
#PSNR
psnr_list = []
for i in range(50):
    try:
        psnr_ouput = tf.image.psnr(y_true_list[i], y_pred_list[i], max_val=1.0)
        psnr_list.append(psnr_ouput)
    except:
        break

In [23]:
len(psnr_list)

50

In [24]:
psnr_list = np.array(psnr_list)

In [25]:
type(psnr_list)

numpy.ndarray

In [26]:
filePath = 'D:/Jupyter/kimna/5. project1/RESRGAN_PSNR.txt'

with open(filePath, 'w') as lf:
    for i in range(len(psnr_list)):
        lf.write(str(psnr_list[i]))
        lf.write('\n')

# SSIM

In [27]:
#SSIM
ssim_list = []
for i in range(len(y_true_list)):
    ssim_ouput = tf.image.ssim(y_true_list[i], y_pred_list[i], max_val=1.0)
    ssim_list.append(ssim_ouput)

In [28]:
ssim_list = np.array(ssim_list)

In [29]:
#SSIM
filePath = 'D:/Jupyter/kimna/5. project1/RESRGAN_SSIM.txt'

with open(filePath, 'w') as lf:
    for i in range(len(ssim_list)):
        lf.write(str(ssim_list[i]))
        lf.write('\n')

### 평가

In [ ]:
clean = clean*255
generate = generate*255

#ref_mat=patch
#res_mat=mask
#print(ref_mat.shape)
#print(res_mat.shape)

In [ ]:
def output_psnr_mse(img_orig, img_out):
    squared_error = np.square(img_orig - img_out)
    mse = np.mean(squared_error)
    psnr = 10 * np.log10(1.0 / mse)
    return psnr

In [ ]:
psnr_output = []
for i in range(len(clean)):
    output = output_psnr_mse(clean[i], generate[i])
    psnr_output.append(output)

In [ ]:
psnr_output

In [ ]:
ref_mat = ref_mat.astype('float') / 255.0
res_mat = res_mat.astype('float') / 255.0

def mean_psnr_srgb(ref_mat, res_mat):
    n_blk, h, w, c = ref_mat.shape
    mean_psnr = 0
    for b in range(n_blk):
        ref_block = ref_mat[b, :, :, :]
        res_block = res_mat[b, :, :, :]
        ref_block = np.reshape(ref_block, (h, w, c))
        res_block = np.reshape(res_block, (h, w, c))
        psnr = output_psnr_mse(ref_block, res_block)
        mean_psnr += psnr
    return mean_psnr / n_blk

#PSNR
mean_psnr = mean_psnr_srgb(ref_mat, res_mat)
print('mean_psnr:')
print(mean_psnr)

In [ ]:
from skimage.metrics import structural_similarity as ssim

def mean_ssim_srgb(ref_mat, res_mat):
    n_blk, h, w, c = ref_mat.shape
    mean_ssim = 0
    for b in range(n_blk):
        ref_block = ref_mat[b, :, :, :]
        res_block = res_mat[b, :, :, :]
        ref_block = np.reshape(ref_block, (h, w, c))
        res_block = np.reshape(res_block, (h, w, c))
        ssim1 = ssim(ref_block, res_block, gaussian_weights=True, use_sample_covariance=False,
                     multichannel=True)
        mean_ssim += ssim1
    return mean_ssim / n_blk

# SSIM
mean_ssim = mean_ssim_srgb(ref_mat, res_mat)
print('mean_ssim:')
print(mean_ssim)

In [ ]:
import shutil

### Haze

In [ ]:
HAZE_BEST_LIST = []

HAZE_BEST_LIST_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/HAZE_BEST_SSIM.txt'

f = open(HAZE_BEST_LIST_path, 'r')
for i in range(704):
    line = f.readline()
    HAZE_BEST_LIST.append(line.replace("\n",""))

In [ ]:
from_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/14. inference_denoise_haze_l/'
to_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/21. best_haze_ssim/'

In [ ]:
for i in range(704):
    num = HAZE_BEST_LIST[i]
    from_image_path = from_path+num+'.jpg'
    to_image_path = to_path+num+'.jpg'
    shutil.copyfile(from_image_path, to_image_path)

### Light

In [ ]:
LIGHT_BEST_LIST = []

LIGHT_BEST_PATH = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/LIGHT_BEST_SSIM.txt'

f = open(LIGHT_BEST_PATH, 'r')
for i in range(371):
    line = f.readline()
    LIGHT_BEST_LIST.append(line.replace("\n",""))

In [ ]:
LIGHT_BEST_LIST[0]

In [ ]:
len(LIGHT_BEST_LIST)

In [ ]:
from_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/15. inference_denoise_light_l/'
to_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/22. best_light_ssim/'
    
for i in range(371):
    num = LIGHT_BEST_LIST[i]
    from_image_path = from_path+num+'.jpg'
    to_image_path = to_path+num+'.jpg'
    shutil.copyfile(from_image_path, to_image_path)

### dark

In [ ]:
DARK_BEST_LIST = []

DARK_BEST_PATH = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/DARK_BEST_SSIM.txt'

f = open(DARK_BEST_PATH, 'r')
for i in range(114):
    line = f.readline()
    DARK_BEST_LIST.append(line.replace("\n",""))

In [ ]:
from_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/13. inference_denoise_dark_l/'
to_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/20. best_dark_ssim/'
    
for i in range(114):
    num = DARK_BEST_LIST[i]
    from_image_path = from_path+num+'.jpg'
    to_image_path = to_path+num+'.jpg'
    shutil.copyfile(from_image_path, to_image_path)